In [10]:
site_to_trace = {"28388": "D704206234688", "28389": "D704206334841", 
                 "28845": "D704206226960", "28191": "D704206340882", 
                 "28891": "D704206234025", "28680": "D704206248003", 
                 "28446": "D704206340944", "28787": "D704206241407", 
                 "28162": "D704206244585", "28163": "D704206240276", 
                 "30308": "D704206246998", "28908": "D704206244447", 
                 "29018": "D704206234860", "28645": "D704206247428", 
                 "28149": "D704206233366", "28102": "D704206228156", 
                 "28643": "D704206243914", "29112": "D704206234181", "29113": "D704206340948", 
                 "29111": "D704206241372", "28740": "D704206234028", "28768": "D704206234179", "27871": "D704206340818", 
                 "28677": "D704206234872", "30310": "D704206268329", "28612": "D704206241417", "28372": "D704206226506",
                 "28517": "D704206340946", "28516": "D704206268314", "28988": "D704206340883", "28175": "D704206226201", 
                 "28611": "D704206223884", "28577": "D704206340913", "28190": "D704206226910", "28346": "D704206226329", 
                 "28347": "D704206234731", "28739": "D704206226534", "28679": "D704206234743", "28328": "D704206226241",
                 "28329": "D704206234197", "28205": "D704206234190", 
                 "28150": "D704206228340", "28610": "D704206226499", "28206": "D704206226974", "28659": "D704206340881"}

In [13]:
import json
import pandas as pd
import os
site_to_zip_code = {}

with open('/vagrant/etl-natgrid-lime/test_data_hourly/maps/site_list.json') as fd:
    jsd = json.load(fd)
    dt = jsd['data']
    for dct in dt:
        site_id, zipcode = dct['site_id'], dct['s_postcode']
        site_to_zip_code[str(site_id)] = zipcode


# Loading the NATGrid hourly data
# 
import json
import numpy as np
import pytz
import pandas 
import os
from eemeter.weather.location import zipcode_to_usaf_station
from eemeter.weather import ISDWeatherSource
from datetime import datetime

#file_name = '/vagrant/etl-natgrid-lime/test_data_hourly/raw_site_data/2017-07-27/28610.json'
#file_name = '/vagrant/etl-natgrid-lime/test_data_hourly/raw_site_data/2017-07-27/28162.json'

format='%Y-%m-%d %H:%M:%S'

def prepare_hrl_data_md(file_name):
    with open(file_name) as fp:
        js_data = json.load(fp)
    
    dt = js_data['data']
    #print "Keys in the Json data ",js_data.keys()
    result = []
    index = []
    energy_consumed= []
    day_hrs = []
    week_day = []
    fname = os.path.basename(file_name)[:-5]
    zipcode = site_to_zip_code[fname]
    if fname not in site_to_trace:
        return None, None
    if not zipcode:
        print 'zipcode not found'
        return None, None
    for energy_data in dt:
        # energy_cs can have none values
        energy_cs = energy_data['energy_consumed']
        if energy_cs is None:
            continue
        energy_cs = float(energy_cs) * 0.001
        if not np.isfinite(energy_cs):
            print  energy_cs
        #print energy_cs
        time_st = energy_data['t_stamp']
        pd_dt_tm = pandas.to_datetime(time_st, format=format, utc=True)
        #day_hrs.append(str( pd_dt_tm.hour))
        #week_day.append(str(pd_dt_tm.dayofweek))
        index.append(pd_dt_tm)
        energy_consumed.append(energy_cs)
        
    dt_index = pandas.DatetimeIndex(index, tz=pytz.utc)
    df = pandas.DataFrame({'value' : np.array(energy_consumed, dtype=np.float16),
                           'project_id':[fname for xx in range(len(energy_consumed))],
                           'trace_id' : [site_to_trace[fname] for xx in range(len(energy_consumed))],
                           'interpretation' : ['ELECTRICITY_CONSUMPTION_SUPPLIED' for xx in range(len(energy_consumed))]
                          }, 
                          
                          index=dt_index)
    df = df[~df.index.duplicated(keep='first')].sort_index()
    
    df = df.asfreq('H')
    if len(df) < 3:
        return None, None
    mid = df.index[len(df.index) / 2 ]
    end = df.index[(len(df.index) / 2 ) + 1]
    project_df = pandas.DataFrame({"project_id" : [fname],
                                   "zipcode" :  [zipcode],
                                   "project_start" :[ mid],
                                   "project_end" : [end]
                                   })
    
    return df, project_df


trace_df_lst = []
project_df_lst = []
for   file in os.listdir('/vagrant/etl-natgrid-lime/test_data_hourly/raw_site_data/2017-07-27/'):
    full_f = os.path.join('/vagrant/etl-natgrid-lime/test_data_hourly/raw_site_data/2017-07-27/', file)
    
    df_t, project_df_t = prepare_hrl_data_md(full_f)
    if df_t is None:
        continue
    trace_df_lst.append(df_t)
    project_df_lst.append(project_df_t)
    #print full_f


traces = pd.concat(trace_df_lst)
traces = traces[np.isfinite(traces['value'])]
projects = pd.concat(project_df_lst)

#print projects
#traces.to_csv("/vagrant/natgrid/traces.csv", index_label='start')
#projects.to_csv("/vagrant/natgrid/projects.csv", index=False)



In [16]:
from eemeter.modeling.models import MovingHourlyAverage
import pandas as pd
base_path = '/vagrant/natgrid'
def split_trace(trace_df, interv_start, intervention_end):
    baseline_period_trace = []
    reporting_period_trace = []
    all_trace = []
    baseline_index = []
    reporting_index = []
    all_trace_index = []
    for index, row in trace_df.iterrows():
        if index < interv_start:
            baseline_period_trace.append(row)
            baseline_index.append(index)
            all_trace.append(row)
            all_trace_index.append(index)
        elif index > intervention_end:
            reporting_period_trace.append(row)
            reporting_index.append(index)
            all_trace.append(row)
            all_trace_index.append(index)
    
    return (pd.DataFrame(baseline_period_trace, index=baseline_index), 
           pd.DataFrame(reporting_period_trace, index=reporting_index),
           pd.DataFrame(all_trace, index=all_trace_index))      


import math
for index, project_df in enumerate(project_df_lst):

    trace_df = trace_df_lst[index]
    trace_id = trace_df['trace_id'][0]
    baseline_trace, reporting_trace, all_trace = split_trace(trace_df, project_df['project_start'][0], project_df['project_end'][0])
    #print baseline_trace, reporting_trace, all_trace
    baseline_trace = baseline_trace.asfreq('H')
    reporting_trace = reporting_trace.asfreq('H')
    all_trace = all_trace.asfreq('H')
    baseline_model = MovingHourlyAverage()
    baseline_model.fit(baseline_trace)
    print "Bseline RMSE", baseline_model.rmse

    reporting_period_pred =baseline_model.predict(reporting_trace) 
    reporting_period_pred_df = pd.DataFrame({
        'actual_usage' : reporting_trace['value'],
        'predicted_usage' : reporting_period_pred,
        'error' : reporting_period_pred - reporting_trace['value'],
        'trace_id' : reporting_trace['trace_id'],
        'site_id' : reporting_trace['project_id']
    }, index=reporting_trace.index)
    
    reporting_period_pred_df.to_csv(os.path.join(base_path, trace_id+"_reporting_pred.csv"), index_label='date')
    
    
    reporting_model = MovingHourlyAverage()
    reporting_model.fit(reporting_trace)
    
    print "REporing RMSE " , reporting_model.rmse

    
    
                
    

Bseline RMSE 0.331110824801
REporing RMSE  0.414034622386
Bseline RMSE 2.3890647689
REporing RMSE  2.73739218677
Bseline RMSE 1.49382817371
REporing RMSE  1.51821110048
Bseline RMSE 1.60994285573
REporing RMSE  1.62886343729
Bseline RMSE 1.44599306784
REporing RMSE  1.44927278099
Bseline RMSE 0.441184007854
REporing RMSE  0.459681026806
Bseline RMSE 1.83704701764
REporing RMSE  2.26415577232
Bseline RMSE 2.10061396118
REporing RMSE  2.42734608799
Bseline RMSE 0.605395448846
REporing RMSE  0.761902086492
Bseline RMSE 0.813772068169
REporing RMSE  0.942503858107
Bseline RMSE 1.30018054712
REporing RMSE  1.3486874265
Bseline RMSE 1.91046237503
REporing RMSE  1.82359868393
Bseline RMSE 0.357316168869
REporing RMSE  0.379452989141
Bseline RMSE 0.467441773715
REporing RMSE  0.410349513062
Bseline RMSE 1.67519250218
REporing RMSE  2.04527467054
Bseline RMSE 2.30663906606
REporing RMSE  2.07544279825
Bseline RMSE 4.66236059297
REporing RMSE  2.8612724678
Bseline RMSE 0.570008418846
REporing RM

In [ ]:
tt = traces[traces.index < ]